## Connect to BigChainDB ##

In [1]:
from bigchaindb_driver import BigchainDB
tokens = {}
tokens['app_id'] = 'e2caa86f'
tokens['app_key'] = '768e14bf51c1caf81b5a1b77dfb8e966'
bdb = BigchainDB('https://test.bigchaindb.com', headers=tokens)

## Creating an Asset in dict format ##

In [2]:
bicycle = {
        'data': {
            'bicycle': {
                'serial_number': 'abcd1234',
                'manufacturer': 'bkfab',
            },
        },
    }

In [6]:
metadata = {'Owner': 'Geoff'}

## Cryptographic Identities Generation ##
-----
Alice and Bob are represented by public/private key pairs. The private key is used to sign transactions, meanwhile the public key is used to verify that a signed transaction was indeed signed by the one who claims to be the signee.


In [4]:
from bigchaindb_driver.crypto import generate_keypair

alice, bob = generate_keypair(), generate_keypair()

## Asset Creation ##
We’re now ready to create the digital asset. First, let’s prepare the transaction:

In [7]:
prepared_creation_tx = bdb.transactions.prepare(
        operation='CREATE',
        signers=alice.public_key,
        asset=bicycle,
        metadata=metadata,
    )

In [8]:
from pprint import pprint
pprint(prepared_creation_tx)

{'asset': {'data': {'bicycle': {'manufacturer': 'bkfab',
                                'serial_number': 'abcd1234'}}},
 'id': None,
 'inputs': [{'fulfillment': {'public_key': 'C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF',
                             'type': 'ed25519-sha-256'},
             'fulfills': None,
             'owners_before': ['C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF']}],
 'metadata': {'Owner': 'Geoff'},
 'operation': 'CREATE',
 'outputs': [{'amount': '1',
              'condition': {'details': {'public_key': 'C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF',
                                        'type': 'ed25519-sha-256'},
                            'uri': 'ni:///sha-256;vncoSpJB8cDCGXOu2TvGH8Oy5K76hHjoBR9FVWdO-i4?fpt=ed25519-sha-256&cost=131072'},
              'public_keys': ['C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF']}],
 'version': '2.0'}


The transaction now needs to be fulfilled by signing it with Alice’s private key:

In [9]:
fulfilled_creation_tx = bdb.transactions.fulfill(
        prepared_creation_tx, private_keys=alice.private_key)

In [10]:
pprint(fulfilled_creation_tx)

{'asset': {'data': {'bicycle': {'manufacturer': 'bkfab',
                                'serial_number': 'abcd1234'}}},
 'id': '72d9e0661ca24a0fcdaaba65ce4648429fefb3a5f2c8fd05625c0b274b5d8595',
 'inputs': [{'fulfillment': 'pGSAIKT64k_rnX83OVFd18_8cu_SlKuO-zVSaM4lWEsAUxcegUDv6lHV-Wk6T_1zGCee9K-1UtIN6kWwBXlVISoNYdu3mMsroBLGaBrfA8_bh2RAQ7gLvNYQ5H0AVfRqI4c7t0kO',
             'fulfills': None,
             'owners_before': ['C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF']}],
 'metadata': {'Owner': 'Geoff'},
 'operation': 'CREATE',
 'outputs': [{'amount': '1',
              'condition': {'details': {'public_key': 'C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF',
                                        'type': 'ed25519-sha-256'},
                            'uri': 'ni:///sha-256;vncoSpJB8cDCGXOu2TvGH8Oy5K76hHjoBR9FVWdO-i4?fpt=ed25519-sha-256&cost=131072'},
              'public_keys': ['C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF']}],
 'version': '2.0'}


Send it over the bigChainDB node

In [11]:
sent_creation_tx = bdb.transactions.send(fulfilled_creation_tx)

In [12]:
sent_creation_tx == fulfilled_creation_tx

True

In [13]:
txid = fulfilled_creation_tx['id']
print(txid)

72d9e0661ca24a0fcdaaba65ce4648429fefb3a5f2c8fd05625c0b274b5d8595


In [16]:
block_height = bdb.blocks.get(txid=fulfilled_creation_tx['id'])
block_height

42343

In [17]:
block = bdb.blocks.retrieve(str(block_height))
pprint(block)

{'height': 42343,
 'transactions': [{'asset': {'data': {'bicycle': {'manufacturer': 'bkfab',
                                                  'serial_number': 'abcd1234'}}},
                   'id': '6b7068c25b2a347f9d1412ee74931f419bc69773c7160cb47c16859a9e95f16e',
                   'inputs': [{'fulfillment': 'pGSAII9_rJbfITEvHsK9bZ3G7V2ZIO6B0An-zKnFU5wZY8irgUAW9xkbkRZIMw-mf3S7xYNTpQJYceb9QnE-zNEyHZzbzyd8UGfz0tpABf44_j8B8EHf_7jmPnaO8lDvxnYOyC0O',
                               'fulfills': None,
                               'owners_before': ['AfADZCmK8jxxQ6xrGqWunz8264a26vTsiG1oG2VCiQYJ']}],
                   'metadata': {'Owner': 'Geoff'},
                   'operation': 'CREATE',
                   'outputs': [{'amount': '1',
                                'condition': {'details': {'public_key': 'AfADZCmK8jxxQ6xrGqWunz8264a26vTsiG1oG2VCiQYJ',
                                                          'type': 'ed25519-sha-256'},
                                              'uri'

Check this:
https://test.bigchaindb.com/api/v1/transactions/72d9e0661ca24a0fcdaaba65ce4648429fefb3a5f2c8fd05625c0b274b5d8595

Alice could retrieve the transaction:

In [15]:
creation_tx = bdb.transactions.retrieve(txid)
asset_id = creation_tx['id']
transfer_asset = {
        'id': asset_id,
    }

Let’s now prepare the transfer transaction:

In [16]:
output_index = 0

output = creation_tx['outputs'][output_index]

transfer_input = {
    'fulfillment': output['condition']['details'],
    'fulfills': {
         'output_index': output_index,
         'transaction_id': creation_tx['id'],
     },
     'owners_before': output['public_keys'],
}


prepared_transfer_tx = bdb.transactions.prepare(
    operation='TRANSFER',
    asset=transfer_asset,
    inputs=transfer_input,
    recipients=bob.public_key)

fulfill it:

In [17]:
fulfilled_transfer_tx = bdb.transactions.fulfill(
        prepared_transfer_tx,
        private_keys=alice.private_key,
    )

In [18]:
pprint(fulfilled_transfer_tx)

{'asset': {'id': '72d9e0661ca24a0fcdaaba65ce4648429fefb3a5f2c8fd05625c0b274b5d8595'},
 'id': '1f62cdbed8bf539b5c45db2098718ab82a85888a5b36c5113c06385c0701ed94',
 'inputs': [{'fulfillment': 'pGSAIKT64k_rnX83OVFd18_8cu_SlKuO-zVSaM4lWEsAUxcegUDOT5kUgw9Vtx_ZuIyH_OdrWxSvSaokFXGOWhGEbXpQUod22vYlbmoS389DH07N0ZDPdUApT-4qBT0514qyqF8F',
             'fulfills': {'output_index': 0,
                          'transaction_id': '72d9e0661ca24a0fcdaaba65ce4648429fefb3a5f2c8fd05625c0b274b5d8595'},
             'owners_before': ['C71kF1JR3BayFQP1GPJKXGwudNT7eN8L8gHYPn1PPxBF']}],
 'metadata': None,
 'operation': 'TRANSFER',
 'outputs': [{'amount': '1',
              'condition': {'details': {'public_key': 'DvY2qFn8KiArtEHt3bQ9s1SLMCxFSfAH51vCCuqMesiP',
                                        'type': 'ed25519-sha-256'},
                            'uri': 'ni:///sha-256;pQNJmsxpMUxkwU_Sbx-Nwx_Rh57O8K5hNi133xrMPqU?fpt=ed25519-sha-256&cost=131072'},
              'public_keys': ['DvY2qFn8KiArtEHt3bQ9s1SLMCx

and finally send it to the connected BigchainDB node:

In [19]:
sent_transfer_tx = bdb.transactions.send_commit(fulfilled_transfer_tx)
sent_transfer_tx == fulfilled_transfer_tx

True

Bob is the new owner:

In [20]:
fulfilled_transfer_tx['outputs'][0]['public_keys'][0] == bob.public_key

True

Alice is the former owner:

In [21]:
fulfilled_transfer_tx['inputs'][0]['owners_before'][0] == alice.public_key

True